# Set up

In [2]:
# 01 Requirment installation

!pip -q install sentence-transformers scikit-learn joblib datasets \
               transformers accelerate
!pip install -q git+https://github.com/bitsandbytes-foundation/bitsandbytes.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
# 02 Imports

# Data handling
import pandas as pd  # For data loading
import numpy as np   # Numeric operations
from datasets import Dataset  # HuggingFace Dataset

# Transformers for NLP
from transformers import (
    pipeline,
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)

# Fine-tuning packages and
from peft import LoraConfig, get_peft_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Utilities
import joblib    # Model serialization, remove if unused
import pickle    # Data serialization, remove if unused

In [4]:
# 04 Emotion detection: zero-shot
zero_shot = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

EMOTIONS_BASED_POSITIVE = {"joy"}
EMOTIONS_BASED_NEGATIVE_NO_RISK = {"sadness", "fear", "stress", "anger", "guilt", "shame", "disgust", "loneliness", "anxiety", "depression"}
MENTAL_HEALTH_NEGATIVE_HIGH_RISK = {"suicidal ideation", "homicidal ideation", "bipolar disorder", "PTSD", "self-harm", "substance abuse", "eating disorder"}

ALL_SITUATIONS = list(EMOTIONS_BASED_POSITIVE | EMOTIONS_BASED_NEGATIVE_NO_RISK | MENTAL_HEALTH_NEGATIVE_HIGH_RISK)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


In [5]:
# 07 LLM loading: two pipelines (gen vs. text_generator) overlap
MODEL_ID = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model     = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID)

gen = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=-1,             # CPU
    max_new_tokens=32,     # short replies
)

Device set to use cpu


In [6]:
# Detect emotion risk
def detect_emotion_risk(text: str) -> str:
    lower = text.lower()
    if re.search(r"\b(i don'?t want to live|want to die|kill myself|end my life|i will jump|going to jump|goodbye letter)\b", lower):
        return "suicidal ideation"
    if re.search(r"\bkill.*(person|someone|them)\b", lower):
        return "homicidal ideation"
    if re.search(r"\b(cut myself|hurt myself|burn myself)\b", lower):
        return "self-harm"
    if re.search(r"\b(binge|purge|skip meals|throw up food)\b", lower):
        return "eating disorder"
    if re.search(r"\b(drink.*to cope|need alcohol|get high)\b", lower):
        return "substance abuse"
    if re.search(r"\b(flashbacks|reliving.*trauma|nightmares)\b", lower):
        return "ptsd"

    result = zero_shot(text, candidate_labels=ALL_SITUATIONS, hypothesis_template="This text indicates {}.")
    return result["labels"][0].lower()

In [7]:
def generate_reply(user_text: str) -> dict:
    emotion_detected = detect_emotion_risk(user_text)

    if emotion_detected in EMOTIONS_BASED_POSITIVE:
        reply = "Congratulations! I’m glad you’re feeling good today. Share your joy with someone if you can—it often grows when shared!"
        emotion_risk = "positive"

    elif emotion_detected in EMOTIONS_BASED_NEGATIVE_NO_RISK:
        # Companion tone suggestions based on specific emotions
        if emotion_detected == "sadness":
            reply = (
                "It sounds like you’re feeling low, and that’s okay. "
                "Maybe try writing down one small thing that brings you comfort, like music or a memory. "
                "You’re not alone—I'm here with you."
            )
        elif emotion_detected == "stress":
            reply = (
                "You seem overwhelmed right now. Try a quick breathing exercise: inhale for 4 seconds, exhale for 6. "
                "Even a short walk or stretch can help reset your mind. I'm right here if you need to pause together."
            )
        elif emotion_detected == "anxiety":
            reply = (
                "Feeling anxious can be intense. You might try progressive muscle relaxation—tense and release each part of your body from your toes to your head. "
                "You’re doing your best, and I’m here beside you."
            )
        elif emotion_detected == "fear":
            reply = (
                "Fear can feel paralyzing. Let’s pause for three deep breaths together. "
                "Then try naming one thing you *can* control right now. I believe in you."
            )
        elif emotion_detected == "guilt":
            reply = (
                "Guilt shows you care, and that’s not a bad thing. "
                "Try writing a gentle letter to yourself, or think about how you'd comfort a friend in your place."
            )
        elif emotion_detected == "shame":
            reply = (
                "Shame can be heavy. Just know that one mistake doesn’t define you. "
                "Recall a time when you overcame something hard—you still carry that strength."
            )
        elif emotion_detected == "disgust":
            reply = (
                "Feeling self-disgust can be painful. You deserve compassion too. "
                "Can you name one neutral or even kind thing about yourself right now?"
            )
        elif emotion_detected == "loneliness":
            reply = (
                "Loneliness can feel like a heavy weight. "
                "Consider sending a message to someone you trust, or joining a simple online group. You’re not invisible—I see you."
            )
        else:
            reply = (
                "I understand you're feeling off right now. A warm drink or a small act of self-kindness can help a little. "
                "Whatever it is, I’ll stay with you."
            )
        emotion_risk = "negative-no-risk"

    elif emotion_detected in MENTAL_HEALTH_NEGATIVE_HIGH_RISK:
        # Specific high-risk resources and suggestions
        if emotion_detected == "suicidal ideation":
            reply = (
                "I'm really concerned you may be having suicidal thoughts. "
                "Please don't go through this alone. Contact Samaritans (UK) at 116 123 or email jo@samaritans.org. "
                "You matter, and help is available 24/7."
            )
        elif emotion_detected == "homicidal ideation":
            reply = (
                "It’s important to get help immediately if you feel like you might harm someone. "
                "Please contact emergency services right away (in the UK, dial 999), or speak to a mental health professional as soon as possible."
            )
        elif emotion_detected == "bipolar disorder":
            reply = (
                "You may be describing symptoms related to bipolar disorder. "
                "Tracking your mood patterns daily and consulting a psychiatrist can help clarify what’s happening and find the right support plan for you."
            )
        elif emotion_detected == "self-harm":
            reply = (
                "I'm sorry you're feeling this way. You're not alone. "
                "It might help to talk to someone you trust or text a crisis support line. "
                "You deserve care and support—even in your darkest moments."
            )
        elif emotion_detected == "ptsd":
            reply = (
                "Reliving traumatic experiences can be really distressing. "
                "Try a grounding technique: name 5 things you can see, 4 you can touch, 3 you can hear. "
                "Consider reaching out to a trauma-informed counselor when you're ready."
            )
        elif emotion_detected == "substance abuse":
            reply = (
                "It sounds like you're using substances to cope, and that’s really hard. "
                "There are support groups and professionals who can guide you toward healing—you're not alone in this."
            )
        elif emotion_detected == "eating disorder":
            reply = (
                "Food and body image struggles are incredibly tough. "
                "You deserve kindness, nourishment, and support. "
                "Talking to a therapist or dietitian who understands eating disorders could really help."
            )
        else:
            reply = (
                "I understand you’re going through a difficult time. "
                "Please consider speaking to a mental health professional or contacting your local support service."
            )
        emotion_risk = "negative-high-risk"

    else:
        reply = (
            "I’m sorry, I didn’t quite understand how you feel. "
            "It might be helpful to talk to a professional or someone you trust. I’ll be here if you want to keep talking."
        )
        emotion_risk = "unknown"

    return {
        "emotion": emotion_detected,
        "emotion_risk": emotion_risk,
        "reply": reply
    }

In [8]:
if __name__ == "__main__":
    while True:
        user = input("Client: ")
        if user.lower() in {"quit", "exit"}:
            break
        result = generate_reply(user)
        print(f"[Emotion: {result['emotion']} | Risk Level: {result['emotion_risk']}]")
        print(result['reply'], "\n")

Client: I feel so sad
[Emotion: sadness | Risk Level: negative-no-risk]
It sounds like you’re feeling low, and that’s okay. Maybe try writing down one small thing that brings you comfort, like music or a memory. You’re not alone—I'm here with you. 

Client: quit


Fine-tuning

In [11]:


df = pd.read_csv("emotion_risk_dataset.csv")

def finetune_from_dataframe(df: pd.DataFrame, label_col: str, label_list: list, output_dir: str,
                            model_name="facebook/bart-large-mnli", peft_r=4, epochs=3, batch_size=8):

    # 1. 標籤轉為索引
    df = df.copy()
    df["label_idx"] = df[label_col].apply(lambda x: label_list.index(x))

    # 2. 切分訓練集與驗證集
    train_df, val_df = train_test_split(df, test_size=0.2, stratify=df[label_col], random_state=42)

    # 3. 轉為 HuggingFace Dataset 格式
    train_ds = Dataset.from_pandas(train_df[["text", "label_idx"]].rename(columns={"label_idx": "labels"}))
    val_ds = Dataset.from_pandas(val_df[["text", "label_idx"]].rename(columns={"label_idx": "labels"}))

    # 4. Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def preprocess(examples):
        return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

    train_ds = train_ds.map(preprocess, batched=True)
    val_ds = val_ds.map(preprocess, batched=True)

    # 5. LoRA 模型初始化
    base = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_list), ignore_mismatched_sizes=True)
    lora_conf = LoraConfig(task_type="SEQ_CLS", r=peft_r, lora_alpha=32, lora_dropout=0.1)
    model = get_peft_model(base, lora_conf)

    # 6. 評估指標
    def compute_metrics(p):
        preds = p.predictions[0].argmax(-1)
        labels = p.label_ids
        prec, rec, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=0)
        acc = accuracy_score(labels, preds)
        return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}

    # 7. 訓練參數
    args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        logging_steps=10,
        save_total_limit=1,
        load_best_model_at_end=False,
        do_eval=True,
        eval_steps=100
    )

    # 8. 微調
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    trainer.train()
    return trainer.evaluate()


In [12]:
# 情緒微調
emotion_labels = sorted(df["emotion"].unique().tolist())
results_emotion = finetune_from_dataframe(df, label_col="emotion", label_list=emotion_labels, output_dir="./emotion-finetuned")

# 風險微調
risk_labels = sorted(df["emotion_risk"].unique().tolist())
results_risk = finetune_from_dataframe(df, label_col="emotion_risk", label_list=risk_labels, output_dir="./risk-finetuned")

print("Emotion classification:", results_emotion)
print("Risk classification:", results_risk)

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-large-mnli and are newly initialized because the shapes did not match:
- classification_head.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([18]) in the model instantiated
- classification_head.out_proj.weight: found shape torch.Size([3, 1024]) in the checkpoint and torch.Size([18, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-11-bc223859b515>:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_name

Step,Training Loss
10,2.887700
20,2.796500
30,2.789100
40,2.725500


Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

<ipython-input-11-bc223859b515>:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,3.159500
20,2.026500
30,2.008500
40,1.849200


Emotion classification: {'eval_loss': 2.680546522140503, 'eval_accuracy': 0.23333333333333334, 'eval_precision': 0.2916666666666667, 'eval_recall': 0.23333333333333334, 'eval_f1': 0.25925925925925924, 'eval_runtime': 56.9001, 'eval_samples_per_second': 0.527, 'eval_steps_per_second': 0.07, 'epoch': 3.0}
Risk classification: {'eval_loss': 1.448003888130188, 'eval_accuracy': 0.4666666666666667, 'eval_precision': 0.47222222222222227, 'eval_recall': 0.4666666666666667, 'eval_f1': 0.3865546218487395, 'eval_runtime': 57.3412, 'eval_samples_per_second': 0.523, 'eval_steps_per_second': 0.07, 'epoch': 3.0}
